## Global Settings and Import

In [1]:
import sys, os
sys.path.append("../../")
import numpy as np
import lightgbm as lgb
import papermill as pm
import pandas as pd
import category_encoders as ce
from tempfile import TemporaryDirectory
from sklearn.metrics import roc_auc_score, log_loss

import import_ipynb
import lightgbm_utils as lgb_utils
#import reco_utils.dataset.criteo as criteo

print("System version: {}".format(sys.version))
print("LightGBM version: {}".format(lgb.__version__))

importing Jupyter notebook from lightgbm_utils.ipynb
System version: 3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]
LightGBM version: 3.0.0


## Parameter Setting

In [2]:

MAX_LEAF = 64
MIN_DATA = 20
NUM_OF_TREES = 100
TREE_LEARNING_RATE = 0.15
EARLY_STOPPING_ROUNDS = 20
METRIC = "auc"
SIZE = "sample"

In [3]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'num_class': 1,
    'objective': "binary",
    'metric': METRIC,
    'num_leaves': MAX_LEAF,
    'min_data': MIN_DATA,
    'boost_from_average': True,
    #set it according to your cpu cores.
    'num_threads': 20,
    'feature_fraction': 0.8,
    'learning_rate': TREE_LEARNING_RATE,
}

## Import Data

In [5]:
all_data = pd.read_csv("final_snack_data.csv")

FileNotFoundError: [Errno 2] File final_snack_data.csv does not exist: 'final_snack_data.csv'

In [30]:
# split data to 3 sets    
length = len(all_data)
train_data = all_data.loc[:0.8*length-1]
valid_data = all_data.loc[0.8*length:0.9*length-1]
test_data = all_data.loc[0.9*length:]

## Basic Usage
### Ordinal Encoding

In [32]:
ord_encoder = ce.ordinal.OrdinalEncoder(cols=cate_cols)

def encode_csv(df, encoder, label_col, typ='fit'):
    if typ == 'fit':
        df = encoder.fit_transform(df)
    else:
        df = encoder.transform(df)
    y = df[label_col].values
    del df[label_col]
    return df, y

train_x, train_y = encode_csv(train_data, ord_encoder, label_col)
valid_x, valid_y = encode_csv(valid_data, ord_encoder, label_col, 'transform')
test_x, test_y = encode_csv(test_data, ord_encoder, label_col, 'transform')

print('Train Data Shape: X: {trn_x_shape}; Y: {trn_y_shape}.\nValid Data Shape: X: {vld_x_shape}; Y: {vld_y_shape}.\nTest Data Shape: X: {tst_x_shape}; Y: {tst_y_shape}.\n'
      .format(trn_x_shape=train_x.shape,
              trn_y_shape=train_y.shape,
              vld_x_shape=valid_x.shape,
              vld_y_shape=valid_y.shape,
              tst_x_shape=test_x.shape,
              tst_y_shape=test_y.shape,))
train_x.head()

Train Data Shape: X: (80000, 39); Y: (80000,).
Valid Data Shape: X: (10000, 39); Y: (10000,).
Test Data Shape: X: (10000, 39); Y: (10000,).



,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,1.0,...,1,1,1,1,1,1,1,1,1,1
1,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,1.0,...,2,2,1,2,2,1,1,2,1,2
2,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,1.0,...,3,3,2,3,3,2,1,3,2,3
3,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,NaN,...,4,4,2,3,4,1,1,4,2,3
4,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,0.0,1.0,...,4,5,2,3,5,1,2,5,2,3


## Create Model

In [33]:
lgb_train = lgb.Dataset(train_x, train_y.reshape(-1), params=params, categorical_feature=cate_cols)
lgb_valid = lgb.Dataset(valid_x, valid_y.reshape(-1), reference=lgb_train, categorical_feature=cate_cols)
lgb_test = lgb.Dataset(test_x, test_y.reshape(-1), reference=lgb_train, categorical_feature=cate_cols)
lgb_model = lgb.train(params,
                      lgb_train,
                      num_boost_round=NUM_OF_TREES,
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                      valid_sets=lgb_valid,
                      categorical_feature=cate_cols)

[LightGBM] [Info] Number of positive: 17958, number of negative: 62042
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.048249 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 38971
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.224475 -> initscore=-1.239776
[LightGBM] [Info] Start training from score -1.239776
[1]	valid_0's auc: 0.723997
Training until validation scores don't improve for 20 rounds
[2]	valid_0's auc: 0.736596
[3]	valid_0's auc: 0.740862
[4]	valid_0's auc: 0.745639
[5]	valid_0's auc: 0.74944
[6]	valid_0's auc: 0.751076
[7]	valid_0's auc: 0.752973
[8]	valid_0's auc: 0.753746
[9]	valid_0's auc: 0.7546
[10]	valid_0's auc: 0.755247
[11]	valid_0's auc: 0.7565
[12]	valid_0's auc: 0.756926
[13]	valid_0's auc: 0.757582
[14]	valid_0's auc: 0.757758
[15]	valid_0's auc: 0.758596
[16]	valid_0's auc: 

In [35]:
test_preds = lgb_model.predict(test_x)
auc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
logloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds), eps=1e-12)
res_basic = {"auc": auc, "logloss": logloss}
print(res_basic)
#pm.record("res_basic", res_basic)

{'auc': 0.7655408801711783, 'logloss': 0.4682583178835999}


## Optimized Usage
### Label-encoding and Binary-encoding

In [41]:
label_col = 'Label'
num_encoder = lgb_utils.NumEncoder(cate_cols, nume_cols, label_col)
train_x, train_y = num_encoder.fit_transform(train_data)
valid_x, valid_y = num_encoder.transform(valid_data)
test_x, test_y = num_encoder.transform(test_data)
del num_encoder
print('Train Data Shape: X: {trn_x_shape}; Y: {trn_y_shape}.\nValid Data Shape: X: {vld_x_shape}; Y: {vld_y_shape}.\nTest Data Shape: X: {tst_x_shape}; Y: {tst_y_shape}.\n'
      .format(trn_x_shape=train_x.shape,
              trn_y_shape=train_y.shape,
              vld_x_shape=valid_x.shape,
              vld_y_shape=valid_y.shape,
              tst_x_shape=test_x.shape,
              tst_y_shape=test_y.shape,))

2020-11-10 15:47:50,024 [INFO] Filtering and fillna features
100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 466.76it/s]
2020-11-10 15:47:51,727 [INFO] Ordinal encoding cate features
2020-11-10 15:47:52,645 [INFO] Target encoding cate features
100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:04<00:00,  6.26it/s]
2020-11-10 15:47:56,804 [INFO] Start manual binary encoding
100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:02<00:00,  9.64it/s]
2020-11-10 15:48:02,996 [INFO] Filtering and fillna features
100%|████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 3249.65it/s]
2020-11-10 15:48:03,121 [INFO] Ordinal encoding cate features
2020-11-10 15:48:03,189 [INFO] Target encoding cate features
100%|██████████████████████████████████████████████████████████████████████████████████| 26/

Train Data Shape: X: (80000, 268); Y: (80000, 1).
Valid Data Shape: X: (10000, 268); Y: (10000, 1).
Test Data Shape: X: (10000, 268); Y: (10000, 1).



## Training and Evaluation

In [42]:
lgb_train = lgb.Dataset(train_x, train_y.reshape(-1), params=params)
lgb_valid = lgb.Dataset(valid_x, valid_y.reshape(-1), reference=lgb_train)
lgb_model = lgb.train(params,
                      lgb_train,
                      num_boost_round=NUM_OF_TREES,
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                      valid_sets=lgb_valid)

[LightGBM] [Info] Number of positive: 17958, number of negative: 62042
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.096549 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15787
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 267
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.224475 -> initscore=-1.239776
[LightGBM] [Info] Start training from score -1.239776
[1]	valid_0's auc: 0.727035
Training until validation scores don't improve for 20 rounds
[2]	valid_0's auc: 0.745243
[3]	valid_0's auc: 0.749993
[4]	valid_0's auc: 0.750781
[5]	valid_0's auc: 0.753203
[6]	valid_0's auc: 0.754548
[7]	valid_0's auc: 0.756143
[8]	valid_0's auc: 0.757718
[9]	valid_0's auc: 0.758371
[10]	valid_0's auc: 0.759261
[11]	valid_0's auc: 0.760498
[12]	valid_0's auc: 0.761402
[13]	valid_0's auc: 0.762118
[14]	valid_0's auc: 0.762916
[15]	valid_0's auc: 0.764051
[16]	valid_0's

In [44]:
test_preds = lgb_model.predict(test_x)
auc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
logloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds), eps=1e-12)
res_optim = {"auc": auc, "logloss": logloss}
print(res_optim)
#pm.record("res_optim", res_optim)

{'auc': 0.7758548016657666, 'logloss': 0.46030887404896165}


## Model saving and loading

In [45]:
with TemporaryDirectory() as tmp:
    save_file = os.path.join(tmp, r'finished.model')
    lgb_model.save_model(save_file)
    loaded_model = lgb.Booster(model_file=save_file)

# eval the performance again
test_preds = loaded_model.predict(test_x)

auc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
logloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds), eps=1e-12)
print({"auc": auc, "logloss": logloss})

{'auc': 0.7758548016657666, 'logloss': 0.46030887404896165}
